<a href="https://colab.research.google.com/github/calemhoffman/softhub/blob/master/python/mona_proposals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [13]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import Scatter
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.io as pio
import math
#pio.templates
color = ['#2CBDFE','#47DBCD','#F3A0F2','#9D2EC5','#661D98','#F5B14C',"#A1A4A7","#100000","#F8F9FA","#A1A4A7"]

# 30F -> 29F

## Notes

In [14]:
# kevin fossez - Z=9 chain nov. 2020
# rebeka - fsu int, z=9 chain nov/dec 2020
# utsuno - sdpf-m, EEdf1 - Dec. 18, 2020
# fortuna - anything on halo nuclei Dec. 18, 2020
# ragnar - any calcs of interest Dec. 18, 2020
# doing sdpf-u locally (0p0h, mix, 2p2h)

## Input data

In [15]:
##@title FSU INT Data #j, energy, number, p-h
f29_fsu_data = np.array([[5/2,-208.378,1,2],
                     [1/2,-208.573,1,2],  
                     [9/2,-207.143,1,2],
                     [5/2,-206.938,2,0],
                     [3/2,-206.374,1,2],
                     [1/2,-203.436,2,0]])

f30_fsu_data = np.array([[2,-206.573,1,2],
                         [3,-206.428,1,2],
                         [4,-206.224,1,2],
                         [4,-205.930,2,0],
                         [2,-205.899,2,0],
                         [6,-205.278,1,0],
                         [6,-205.806,2,2]])


# 2.145], #f30 (3-)
#                  [1.5,2.349],[2.5,2.349], #f30 (4-)
#                  [1.5,2.767],[2.5,2.767], #f30 (6-)
#                  [1.5,2.884],[2.5,2.884], #f30 (2-)
#                  #[1.5,3.069],[2.5,3.069]
f_ex = pd.DataFrame({'state':['2<sup>-</sup>','3<sup>-</sup>','4<sup>-</sup>',
                             '6<sup>-</sup>','2<sup>-</sup><sub>2</sub>','1<sup>-</sup>',
                              '1<sup>+</sup>'],
                    'fsu':[0.0,0.145,0.349,0.767,0.884,1.069,None],
                     'sdpfm':[0.304,None,0.00,0.259,None,None,None],
                     'EEdf1':[0.00,0.177,None,None,None,None,0.124]})
f_ex

#     Exp.    EEdf1   SDPF-M 
# 19F -20.182  -20.131  -23.936
# 21F -34.885  -33.706  -38.184
# 23F -47.667  -44.650  -50.731
# 25F -55.759  -52.473  -59.098
# 27F -57.800  -56.333  -62.076
# 29F -59.242  -58.644  -63.105
# 30F    =     -57.913  -61.828
# 31F    =     -59.880  -62.825
# 32F    =     -57.882  -61.006   
# 33F    =     -59.416  -61.569

f_binding = pd.DataFrame({'interaction': ['fsu','sdpf-m','gsm','gsm-fht','gsm-eft','hf-mbvt','sdpf-u','eedf1'],
                      'f25_be': [0.0,-25.238,-21.674,-14.437,-14.350,-14.437,0.0,-52.473],
                      'f26_be': [-204.920,-25.473,-22.809,-15.531,-15.280,-15.367,-57.4202,None], #0p0h
                      'f27_be': [-206.953,-28.136,-24.533,-17.610,-16.975,-16.943,-59.8226,-56.333], #0p0h
                      'f28_be': [-206.517,-27.548,-24.728,-17.347,-17.260,-17.140,-59.7417,None], #1p1h
                      'f29_be': [-208.378,-29.115,-25.316,-18.770,-18.650,-18.343,-60.685,-58.644], #2p2h
                      'f30_be': [-206.573,-27.822,-22.104,-18.179,-18.540,-16.811,-60.0575,-57.913], #2p2h<0p0h??
                      'f31_be': [-207.596,-28.841,-23.044,-18.803,-18.781,-17.315,-61.45,-59.880], #0p0h
                      'f32_be': [-205.887,-27.039,-15.251,0.0,0.0,0.0,0.0,-57.882]}) #0p0h
f_binding
f_ex

,state,fsu,sdpfm,EEdf1
0,2<sup>-</sup>,0.000,0.304,0.000
1,3<sup>-</sup>,0.145,NaN,0.177
2,4<sup>-</sup>,0.349,0.000,NaN
3,6<sup>-</sup>,0.767,0.259,NaN
4,2<sup>-</sup><sub>2</sub>,0.884,NaN,NaN
5,1<sup>-</sup>,1.069,NaN,NaN
6,1<sup>+</sup>,NaN,NaN,0.124


In [16]:
f29_fsu_df = pd.DataFrame({'j':f29_fsu_data[:,0], 'e':f29_fsu_data[:,1],
                       'n':f29_fsu_data[:,2], 'p-h':f29_fsu_data[:,3]})
f30_fsu_df = pd.DataFrame({'j':f30_fsu_data[:,0], 'e':f30_fsu_data[:,1],
                       'n':f30_fsu_data[:,2], 'p-h':f30_fsu_data[:,3]})
num_rows, num_cols = f29_fsu_data.shape
f29_x_data = np.zeros(num_rows)
num_rows, num_cols = f30_fsu_data.shape
f30_x_data = np.ones(num_rows)
#print(f30_fsu_df.iat[0,1])
f30_fsu_df

,j,e,n,p-h
0,2.0,-206.573,1.0,2.0
1,3.0,-206.428,1.0,2.0
2,4.0,-206.224,1.0,2.0
3,4.0,-205.930,2.0,0.0
4,2.0,-205.899,2.0,0.0
5,6.0,-205.278,1.0,0.0
6,6.0,-205.806,2.0,2.0


In [17]:
#Experimental data
#be/a 
f_exp_be = pd.DataFrame({'f25_be' : [25.012167727,103.535E-6],
                         'f26_be' : [26.020020392,119.377E-6],
                         'f27_be' : [27.027322000,418.500E-6],
                         'f28_be' : [28.036223095,421.928E-6],
                         'f29_be' : [29.043103000,564.000E-6],
                         'f30_be' : [0,0],
                         'f31_be' : [0,0],
                         'f32_be' : [0,0]})

amu=931.494
n=1.00866491582
num_rows,num_cols = f_exp_be.shape
f_Sn_exp = pd.DataFrame(0.0,index=range(num_rows*2),columns=range(num_cols))
for i in range(num_cols-1):
  if f_exp_be.iat[0,i+1] != 0:
    #print((f_exp_be.iat[0,i+1] - f_exp_be.iat[0,i] - n)*amu)
    f_Sn_exp.iat[0,i+1] = (f_exp_be.iat[0,i+1] - f_exp_be.iat[0,i] - n)*amu
    f_Sn_exp.iat[1,i+1] = (math.sqrt(f_exp_be.iat[1,i+1]*f_exp_be.iat[1,i+1]+
                          f_exp_be.iat[1,i]*f_exp_be.iat[1,i]))*amu
    if f_exp_be.iat[0,i+2] != 0:
      #print((f_exp_be.iat[0,i+2] - f_exp_be.iat[0,i] - 2.0*n)*amu)
      f_Sn_exp.iat[2,i+2] = (f_exp_be.iat[0,i+2] - f_exp_be.iat[0,i] - 2.0*n)*amu
      f_Sn_exp.iat[3,i+2] = (math.sqrt(f_exp_be.iat[1,i+2]*f_exp_be.iat[1,i+2]+
                            f_exp_be.iat[1,i]*f_exp_be.iat[1,i]))*amu

f_Sn_exp = f_Sn_exp.T
f_Sn_exp = f_Sn_exp.rename(columns={0:'Sn',1:'Sn_err',2:'S2n',3:'S2n_err'})
f_Sn_exp = f_Sn_exp.drop([0,1])
f_Sn_exp = f_Sn_exp.rename(index={2:'<sup>27</sup>F',3:'<sup>28</sup>F',4:'<sup>29</sup>F',5:'<sup>30</sup>F',6:'<sup>31</sup>F'})
f_Sn_exp = f_Sn_exp*(-1.0)


## Plots

### Decay

In [18]:
markSize=100
f29_trace = go.Scatter(
    x=f29_x_data,           # x-coordinates of trace
    y=f29_fsu_df['e'],          # y-coordinates of trace
    #name='fsu int.',
    mode='markers + text',   # scatter mode
    marker = dict(size = markSize, symbol = 141), # Add symbol here!
     textfont=dict(color='black',size=18,family='Times New Roman')
    )

f30_trace = go.Scatter(
    x=f30_x_data,           # x-coordinates of trace
    y=f30_fsu_df['e'],          # y-coordinates of trace
    mode='markers + text',   # scatter mode
    marker = dict(size = markSize, symbol = 141), # Add symbol here!
     textfont=dict(color='black',size=18,family='Times New Roman')
    )
data = [f29_trace,f30_trace]

fig = go.Figure(data=data)

fig.update_layout(
    title='fsu interaction',
    width=600,
    height=300,
    margin=dict(l=35, r=35, t=35, b=35),
    #paper_bgcolor="LightSteelBlue",
    xaxis=dict(title='isotope',
               range=[-0.5,1.5],
               tickmode='array',tickvals=[0,1],ticktext=['<sup>29</sup>F','<sup>30</sup>F']
               ),
    yaxis=dict(title='binding energy [MeV]', range=[-209,-205]),
    showlegend=False,
    annotations=[dict(x=0.3, y=f29_fsu_data[0,1],ax=0.7, ay=f30_fsu_data[0,1],
                   xref="x",yref="y",axref="x",ayref="y",
                   showarrow=True,arrowhead=3,text=None),
                dict(x=0.3, y=f29_fsu_data[1,1],ax=0.7, ay=f30_fsu_data[0,1],
                   xref="x",yref="y",axref="x",ayref="y",
                   showarrow=True,arrowhead=3,text=None)],
                  template="plotly_white"
    )

fig.show()

### Excite

In [19]:
markSize=100
f30_trace = go.Scatter(
    x=f30_x_data,           # x-coordinates of trace
    y=f30_fsu_df['e']-f30_fsu_df.iat[0,1]+0.01,          # y-coordinates of trace
    mode='markers + text',   # scatter mode
    marker = dict(size = markSize, symbol = 141), # Add symbol here!
     textfont=dict(color='black',size=18,family='Times New Roman'),
    #text=f30_fsu_df['e']-f30_fsu_df.iat[0,1]
    )
data = [f30_trace]
fig = go.Figure(data=data)
fig.update_layout(
    title='fsu interaction',
    width=300,
    height=300,
    margin=dict(l=35, r=35, t=35, b=35),
    #paper_bgcolor="LightSteelBlue",
    xaxis=dict(title='',
               range=[0.5,1.5],
               tickmode='array',tickvals=[1],ticktext=['<sup>30</sup>F'],
               showline=False,showgrid=False
               ),
    yaxis=dict(title='excitation energy [MeV]', range=[0,2]),
    showlegend=False,
    template="plotly_dark"
    )

fig.show()

### Sn systematics

In [20]:
f_Sn = pd.DataFrame()
f_Sn['<sup>27</sup>F'] = f_binding['f27_be']-f_binding['f26_be']
f_Sn['<sup>28</sup>F'] = f_binding['f28_be']-f_binding['f27_be']
f_Sn['<sup>29</sup>F'] = f_binding['f29_be']-f_binding['f28_be']
f_Sn['<sup>30</sup>F'] = f_binding['f30_be']-f_binding['f29_be']
f_Sn['<sup>31</sup>F'] = f_binding['f31_be']-f_binding['f30_be']
f_S2n = pd.DataFrame()
f_S2n['<sup>28</sup>F'] = f_binding['f28_be']-f_binding['f26_be']
f_S2n['<sup>29</sup>F'] = f_binding['f29_be']-f_binding['f27_be']
f_S2n['<sup>30</sup>F'] = f_binding['f30_be']-f_binding['f28_be']
f_S2n['<sup>31</sup>F'] = f_binding['f31_be']-f_binding['f29_be']
f_Sn = f_Sn.T
f_Sn = f_Sn.rename(columns={0: 'fsu', 1:'sdpf-m', 2:'gsm', 
                            3:'gsm-fht', 4:'gsm-eft', 5:'hf-mbvt', 
                            6:'sdpf-u', 7:'eedf1'})
f_S2n = f_S2n.T
f_S2n = f_S2n.rename(columns={0: 'fsu', 1:'sdpf-m', 2:'gsm', 
                              3:'gsm-fht', 4:'gsm-eft', 5:'hf-mbvt', 
                              6:'sdpf-u', 7:'eedf1'})

f_Sn['imsrg'] = [-1.194,0.199,-0.796,1.293,0.00] #-1.493,-0.896
f_S2n['imsrg'] = [30.348-30.945,30.547-30.945,31.841-30.945,31.841-30.945]
print(f_S2n)
print(f_Sn)
f_Sn = f_Sn*(-1.0)
f_S2n = f_S2n*(-1.0)

                  fsu  sdpf-m    gsm  gsm-fht  ...  hf-mbvt  sdpf-u  eedf1  imsrg
<sup>28</sup>F -1.597  -2.075 -1.919   -1.816  ...   -1.773 -2.3215    NaN -0.597
<sup>29</sup>F -1.425  -0.979 -0.783   -1.160  ...   -1.400 -0.8624 -2.311 -0.398
<sup>30</sup>F -0.056  -0.274  2.624   -0.832  ...    0.329 -0.3158    NaN  0.896
<sup>31</sup>F  0.782   0.274  2.272   -0.033  ...    1.028 -0.7650 -1.236  0.896

[4 rows x 9 columns]
                  fsu  sdpf-m    gsm  gsm-fht  ...  hf-mbvt  sdpf-u  eedf1  imsrg
<sup>27</sup>F -2.033  -2.663 -1.724   -2.079  ...   -1.576 -2.4024    NaN -1.194
<sup>28</sup>F  0.436   0.588 -0.195    0.263  ...   -0.197  0.0809    NaN  0.199
<sup>29</sup>F -1.861  -1.567 -0.588   -1.423  ...   -1.203 -0.9433    NaN -0.796
<sup>30</sup>F  1.805   1.293  3.212    0.591  ...    1.532  0.6275  0.731  1.293
<sup>31</sup>F -1.023  -1.019 -0.940   -0.624  ...   -0.504 -1.3925 -1.967  0.000

[5 rows x 9 columns]


In [65]:
Headers = f_Sn.columns.values.tolist()
Indexes = f_Sn.index.values.tolist()
trace = []
num_rows,num_cols = f_Sn.shape
# colnum = [4,3,6,7,1,8,5,0,2] #2
colnum = [7,4,8,1,0,6,2,3,5]
colors_new = px.colors.sequential.Plasma
for i in range(num_cols):
  trace.append(go.Scatter(x=f_Sn.index,
                          y = f_Sn[Headers[i]],
                          mode = 'lines', 
                          name = Headers[i],
                          marker=dict(color=colors_new[colnum[i]]))
  )
trace.append(go.Scatter(
    x=f_Sn_exp.index, #columns[:-1],           # x-coordinates of trace
    y=f_Sn_exp['Sn'],          # y-coordinates of trace
    mode='markers',   # scatter mode
    marker = dict(size = 10, color = 'white'),
    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=f_Sn_exp['Sn_err'],
            visible=True),
    name='exp'
    )
)
fig = go.Figure(data=trace)
fig.update_layout(
    title='',
    width=800,
    height=350,
    margin=dict(l=35, r=35, t=35, b=35),
    xaxis=dict(title='',
               range=[-0.5,4.5],
               tickmode='array',tickvals=[0,1,2,3],
               ticktext=Indexes,
               showline=True,showgrid=True
               ),
    yaxis=dict(title='S<sub>n</sub> [MeV]', 
               range=[-4,4.0]
               ),
    showlegend=True,
    template="plotly_dark",
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
  )
)

fig.show()

In [67]:
Headers = f_S2n.columns.values.tolist()
Indexes = f_S2n.index.values.tolist()
trace = []
num_rows,num_cols = f_S2n.shape
colnum = [7,4,8,1,0,6,2,3,5]
colors_new = px.colors.sequential.Plasma
for i in range(num_cols):
  trace.append(go.Scatter(x=f_S2n.index,
                          y = f_S2n[Headers[i]],
                          mode = 'lines', 
                          name = Headers[i],
                          marker=dict(color=colors_new[colnum[i]]))
  )
trace.append(go.Scatter(
    x=f_Sn_exp.index, #columns[:-1],           # x-coordinates of trace
    y=f_Sn_exp['S2n'],          # y-coordinates of trace
    mode='markers',   # scatter mode
    marker = dict(size = 10, color = 'white'),
    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=f_Sn_exp['S2n_err'],
            visible=True),
    name='exp'
    )
)
fig = go.Figure(data=trace)
fig.update_layout(
    title='',
    width=800,
    height=350,
    margin=dict(l=35, r=35, t=35, b=35),
    xaxis=dict(title='',
               range=[-0.5,3.5],
               tickmode='array',tickvals=[0,1,2,3],
               ticktext=Indexes,
               showline=True,showgrid=True
               ),
    yaxis=dict(title='S<sub>2n</sub> [MeV]', 
               range=[-3,4.0]
               ),
    showlegend=True,
    template="plotly_dark",
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
  )
)

fig.show()

### 0p0h vs. 2p2h


In [23]:
#fsu, sdpfu 0p0h vs. 2p2h
#x still isotope, y normalize BE
f_ph = pd.DataFrame({'isotope': ['26F','27F','28F',
                                 '29F','30F','31F',
                                 '32F'],
                      'fsu0p0h': [-204.92,-206.953,-206.137,
                                  -206.938,-205.930,-207.596,
                                  -205.887,],
                      'fsu2p2h': [-200.204,-205.083,-206.23,
                                  -208.378,-206.573,-206.181,
                                  -203.337],
                      'fsu1p1h': [0.0,0.0,-206.517,0.0,-207.311,0.0,0.0],
                      'sdpfu0p0h': [-57.4202,-59.8226,-58.9684,
                                    -59.8842,-60.0575,-61.45,
                                    -61.3554],
                      'sdpfu2p2h': [-51.60,-56.661,-58.0042,
                                    -60.6854,-59.916,-59.8525,
                                    -58.3978],
                      'sdpfu1p1h': [0,0,-59.7417,
                                    0,-60.1213,0,
                                    -59.9403],
                     'sdpfm': [-25.473,-28.136,-27.548,
                               -29.115,-27.822,-28.841,
                               -27.039]})

norm = [25.473,204.92,57.4202,
        29.115,208.378,60.6854]

fig1 = go.Figure()
fig2 = go.Figure()
fig1.add_trace(go.Scatter(x=f_ph['isotope'], y=f_ph['sdpfm']+norm[3],
                         mode='lines',name='sdpf-m',
                          line=dict(color=color[3])))
fig1.add_trace(go.Scatter(x=f_ph['isotope'], y=f_ph['fsu0p0h']+norm[4],
                         mode='markers+lines',name='fsu 0p-0h',
                          line=dict(color=color[0])))
fig1.add_trace(go.Scatter(x=f_ph['isotope'], y=f_ph['fsu2p2h']+norm[4],
                         mode='markers+lines',name='fsu 2p-2h',
                          line=dict(color=color[2])))
fig1.add_trace(go.Scatter(x=f_ph['isotope'], y=f_ph['fsu1p1h']+norm[4],
                         mode='markers',name='fsu 1p-1h',
                          line=dict(color=color[5])))
fig2.add_trace(go.Scatter(x=f_ph['isotope'], y=f_ph['sdpfm']+norm[3],
                         mode='lines',name='sdpf-m',
                          line=dict(color=color[3])))
fig2.add_trace(go.Scatter(x=f_ph['isotope'], y=f_ph['sdpfu0p0h']+norm[5],
                         mode='markers+lines',name='sdpf-u 0p-0h',
                          line=dict(color=color[0])))
fig2.add_trace(go.Scatter(x=f_ph['isotope'], y=f_ph['sdpfu2p2h']+norm[5],
                         mode='markers+lines',name='sdpf-u 2p-2h',
                          line=dict(color=color[2])))
fig2.add_trace(go.Scatter(x=f_ph['isotope'], y=f_ph['sdpfu1p1h']+norm[5],
                         mode='markers',name='sdpf-u 1p-1h',
                          line=dict(color=color[5])))
fig1.update_yaxes(title='binding energy [MeV]',range=[-2,6])
fig1.update_layout(font_size=14,title='Xp-Xh refers to particles across N=20 (sd-pf) gap',
                  width=800,height=350,
                  template="plotly_dark",
                   legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="center",x=0.5))
fig.update_layout()
fig2.update_yaxes(title='binding energy [MeV]',range=[-2,6])
fig2.update_layout(font_size=14,
                  width=800,height=350,
                  template="plotly_dark",
                   legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="center",x=0.5))
fig1.show()
fig2.show()


### 30F level compare

In [79]:
#do decay from 30 to 29 for all ...
#draw line with f_Sn differences
#then annote for arrows
#29f at Y=0
import plotly.io as pio

# naming a layout theme for future reference
pio.templates["google"] = go.layout.Template(
    layout_colorway=['#4285F4', '#DB4437', '#F4B400', '#0F9D58',
                     '#185ABC', '#B31412', '#EA8600', '#137333',
                     '#d2e3fc', '#ceead6']
)

# setting Google color palette as default
pio.templates.default = "google"

colnum = [4,3,6,7,1,5,0] #2
offset = [0.05,-0.05,0.05,0.05,0.05,0.05,0.05]
import plotly.express as px
colors_new = px.colors.sequential.Plasma
fig = go.Figure()
fig.add_shape(type="rect",x0=0.75, y0=1.1, x1=1.25, y1=1.88,
    line=dict(width=0),
    fillcolor='Grey',opacity=0.3
)
fig.add_shape(type="rect",x0=0.25, y0=1.22, x1=0.75, y1=2.1,
    line=dict(width=0),
    fillcolor='Grey',opacity=0.3
)
fig.add_trace(go.Scatter(x=[0.25,1.25],y=[0,0],mode='lines',marker=dict(color='black'),
                         name='',showlegend=False))
fig.add_trace(go.Scatter(x=[0.25,1.25],y=[1.08,1.08],mode='lines',marker=dict(color='black'),
                         name='',showlegend=False))
fig.add_trace(go.Scatter(x=[0.25,0.75],y=[1.66,1.66],mode='lines',marker=dict(color='black'),
                         name='',showlegend=False))
fig.add_trace(go.Scatter(x=[0.75,1.25],y=[1.44,1.44],mode='lines',marker=dict(color='black'),
                         name='',showlegend=False))
for i in range(len(colnum)):
  fig.add_trace(go.Scatter(x=[2,3],y=[-1.0*f_Sn.iloc[3,colnum[i]],-1.0*f_Sn.iloc[3,colnum[i]]],
                           mode='lines',marker=dict(color=colors_new[8-i]),
                           name=f_Sn.columns[colnum[i]]))
  # fig.add_trace(go.Scatter(x=[3.75,4.75],y=[-1.0*f_S2n.iloc[3,colnum[i]],-1.0*f_S2n.iloc[3,colnum[i]]],
  #                          mode='lines',marker=dict(color=colors_new[8-i]),
  #                          name=f_S2n.columns[colnum[i]]))
  fig.add_annotation(ax=2,ay=-1.0*f_Sn.iloc[3,colnum[i]],xref="x",yref="y",
                   x=1.25,y=0,axref="x",ayref="y",text="",showarrow=True,
                    arrowhead=2,arrowsize=1,arrowwidth=2,arrowcolor=colors_new[8-i])
  if i>3 :
    fig.add_annotation(x=2,y=-1.0*f_Sn.iloc[3,colnum[i]]+offset[i],xref="x",yref="y",
                    text="{:.2f}[{:.2f}] ".format(-1.0*f_Sn.iloc[3,colnum[i]],
                                                  -1.0*f_Sn.iloc[3,colnum[i]]-1.08),
                       showarrow=False,xanchor='left')
    fig.add_annotation(ax=2,ay=-1.0*f_Sn.iloc[3,colnum[i]],xref="x",yref="y",
                   x=1.25,y=1.08,axref="x",ayref="y",text="",showarrow=True,
                    arrowhead=2,arrowsize=1,arrowwidth=2,arrowcolor=colors_new[8-i])
  else:
    fig.add_annotation(x=2,y=-1.0*f_Sn.iloc[3,colnum[i]]+offset[i],xref="x",yref="y",
                    text="{:.2f}".format(-1.0*f_Sn.iloc[3,colnum[i]]),
                    showarrow=False,xanchor='left')
fig.add_annotation(x=0.5,y=1.71,xref='x',yref='y',text='S<sub>n</sub>=1.66(44)',
                   showarrow=False,xanchor='center',
                   font=dict(size=12))
fig.add_annotation(x=1.0,y=1.49,xref='x',yref='y',text='S<sub>n</sub>=1.44(43)',
                   showarrow=False,xanchor='center',
                   font=dict(size=12))
fig.update_layout(font_size=14,width=1000,height=600,
                  legend=dict(orientation="h",yanchor="bottom",
                              y=1.02,xanchor="center",x=0.5))
fig.update_yaxes(title='relative decay energy [MeV]',range=[-0.25,2.2],zeroline=False)
fig.update_xaxes(ticktext=['<sup>29</sup>F','<sup>30</sup>F','<sup>31</sup>F'],tickvals=[0.75,2.5,4.25],
                 range=[-0.25,3.750])

fig.show()
f_Sn


,fsu,sdpf-m,gsm,gsm-fht,gsm-eft,hf-mbvt,sdpf-u,eedf1,imsrg
<sup>27</sup>F,2.033,2.663,1.724,2.079,1.695,1.576,2.4024,NaN,1.194
<sup>28</sup>F,-0.436,-0.588,0.195,-0.263,0.285,0.197,-0.0809,NaN,-0.199
<sup>29</sup>F,1.861,1.567,0.588,1.423,1.390,1.203,0.9433,NaN,0.796
<sup>30</sup>F,-1.805,-1.293,-3.212,-0.591,-0.110,-1.532,-0.6275,-0.731,-1.293
<sup>31</sup>F,1.023,1.019,0.940,0.624,0.241,0.504,1.3925,1.967,-0.000


# Experiment

## Population and decay scheme


In [ ]:
#hand draw 31Ne -> 30F -> 29F -> 28F if necc.
#include C2S from FSU / SDPFU for now
set_color = [6,0,2,2,2,
             6,6,6,6,6,
             1,3,1,3,3,
             1,3,6,6,6]
#set 30F g.s. at = 2
ne30=4
data = np.array([[0,ne30],[1,ne30], #ne30
                 [1.01,ne30],[1.49,2], #ne30 2 f30 1
                 [1.01,ne30],[1.45,2], #ne30 2 f30 1
                 [1.01,ne30],[1.49,2.145], #ne30 2 f30 1
                 [1.01,ne30],[1.49,2.349], #ne30 2 f30 1
                 #[1.01,ne30],[1.49,3.069], #ne30 2 f30 1
                 [1.5,2.000],[2.5,2.000], #f30 (2-)
                 [1.5,2.145],[2.5,2.145], #f30 (3-)
                 [1.5,2.349],[2.5,2.349], #f30 (4-)
                 [1.5,2.767],[2.5,2.767], #f30 (6-)
                 [1.5,2.884],[2.5,2.884], #f30 (2-)
                 #[1.5,3.069],[2.5,3.069], #f30 (1-)
                 [2.51,2],[2.95,0.195], #f30 2 f29
                 [2.51,2],[2.99,0.195], #f30 2 f29
                 [2.51,2.145],[2.95,0.195], #f30 2 f29
                 [2.51,2.145],[2.99,0.195], #f30 2 f29
                 [2.51,2.349],[2.95,0.195], #f30 2 f29
                 [2.51,2.0],[2.95,1.1], #f30 2 f29
                 [2.51,2.349],[2.95,1.1], #f30 2 f29
                 [3,0.195],[4,0.195], #f29 g,s
                 [3,1.1],[4,1.1] #f29 1ex
                 ])
df_data = pd.DataFrame(data, columns=['x','y'])
fig = go.Figure()
for i in range(19):
  fig.add_trace(go.Scatter(mode='lines',x=df_data.x[i*2:i*2+2],y=df_data.y[i*2:i*2+2],
                line=dict(color=color[set_color[i]]))
                )
fig.update_layout(template='plotly_white',width=800,height=600,
                  # annotations=[dict(x=1,y=5,
                  #                   ax=1.5, ay=3,
                  #  xref="x",yref="y",axref="x",ayref="y",
                  #  showarrow=True,arrowhead=3,text=None)]
                  )
fig.show()


## FWHM Resolution Fit

In [ ]:
from scipy.optimize import curve_fit
from numpy import arange

# define the true objective function
def objective(x, a, b):
	return a * x**b

x = np.array([0.05,0.1,0.2,0.3,0.4,0.5,0.75,1.0,1.25,1.5])
y = np.array([0.0328,0.0487,0.070,0.0865,0.1023,0.113,0.144,0.167,0.189,0.208])

y *= 2.355
popt, _ = curve_fit(objective, x, y)
# summarize the parameter values
a, b = popt
print('y = %.2f * x^%.2f' % (a, b))
print(objective(0.5,a,b))
# define a sequence of inputs between the smallest and largest known inputs
#x_line = arange(min(x), max(x), 0.001)
x_line = arange(0, 2, 0.001)
# calculate the output for the range
y_line = objective(x_line, a, b)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y,
                      mode='markers', name='data'))
fig.add_trace(go.Scatter(x=x_line,y=y_line,
                         mode='lines', name = 'fit'))
fig.update_layout(title='FWHM for 188 mg/cm2 Be target and 8 m', width=800,height=400,
                  template="plotly_dark")
fig.update_yaxes(title='FWHM [MeV]')
fig.update_xaxes(title='relative decay energy [MeV]')
fig.show()